In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy qiskit-addon-cutting

*Estimation d'utilisation : deux minutes sur un processeur Eagle (REMARQUE : il s'agit uniquement d'une estimation. Votre temps d'exécution peut varier.)*

## Contexte

Dans ce notebook, nous considérons la simulation d'une chaîne périodique de qubits où une opération à deux qubits est appliquée entre chaque paire de qubits adjacents, y compris entre le premier et le dernier. Les chaînes périodiques se retrouvent fréquemment dans les problèmes de physique et de chimie tels que les modèles d'Ising et la simulation moléculaire.

Les dispositifs IBM Quantum&reg; actuels sont planaires. Il est possible d'intégrer certaines chaînes périodiques directement sur la topologie lorsque les premier et dernier qubits sont voisins. Cependant, pour des problèmes suffisamment grands, les premier et dernier qubits peuvent être éloignés, nécessitant ainsi de nombreuses portes SWAP pour l'opération à 2 qubits entre ces deux qubits. Ce type de problème à conditions aux limites périodiques a été étudié dans <a href="https://arxiv.org/abs/2402.17833">cet article</a>.

Dans ce notebook, nous montrons l'utilisation de la découpe de circuits pour traiter un problème de chaîne périodique à grande échelle où les premier et dernier qubits ne sont pas voisins. La découpe de cette connectivité longue distance évite les portes SWAP supplémentaires au prix de l'exécution de plusieurs instances du circuit, ainsi que d'un post-traitement classique. En résumé, la découpe peut être intégrée pour calculer logiquement les opérations à 2 qubits longue distance. Autrement dit, cette approche conduit à une augmentation effective de la connectivité de la carte de couplage, entraînant ainsi un nombre réduit de portes SWAP.

Notez qu'il existe deux types de découpes : couper le fil d'un circuit (appelé `wire cutting`), ou remplacer une porte à 2 qubits par plusieurs opérations à un seul qubit (appelé `gate cutting`). Dans ce notebook, nous nous concentrerons sur le gate cutting. Pour plus de détails sur le gate cutting, consultez les <a href="https://qiskit.github.io/qiskit-addon-cutting/explanation/index.html">documents explicatifs</a> de `qiskit-addon-cutting`, ainsi que les références correspondantes. Pour plus de détails sur le wire cutting, consultez le tutoriel [Découpe de fils pour l'estimation des valeurs d'espérance](/tutorials/wire-cutting), ou les tutoriels de <a href='https://qiskit.github.io/qiskit-addon-cutting/tutorials/index.html'>qiskit-addon-cutting</a>.

## Prérequis

Avant de commencer ce tutoriel, assurez-vous d'avoir installé les éléments suivants :

- Qiskit SDK v1.2 ou ultérieur (`pip install qiskit`)
- Qiskit Runtime v0.3 ou ultérieur (`pip install qiskit-ibm-runtime`)
- Addon de découpe de circuits Qiskit v.9.0 ou ultérieur (`pip install qiskit-addon-cutting`)

## Configuration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (
    BasisTranslator,
    Optimize1qGatesDecomposition,
)
from qiskit.circuit.equivalence_library import (
    SessionEquivalenceLibrary as sel,
)
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.result import sampled_expectation_value
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal

from qiskit_addon_cutting import (
    cut_gates,
    generate_cutting_experiments,
    reconstruct_expectation_values,
)


from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2, SamplerOptions, Batch

## Étape 1 : Mapper les entrées classiques vers un problème quantique
Ici, nous allons générer un circuit TwoLocal et définir quelques observables.

<ul>
    <li>Entrée : Paramètres pour créer un circuit</li>
    <li>Sortie : Circuit abstrait et observables</li>
</ul>
Nous considérons une `carte d'intrication` à efficacité matérielle pour le circuit TwoLocal avec une connectivité périodique entre les dernier et premier qubits de la `carte d'intrication`. Cette interaction longue distance peut entraîner des portes SWAP supplémentaires lors de la transpilation, augmentant ainsi la profondeur du circuit.
#### Sélection du backend et du placement initial

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

Pour ce notebook, nous considérons une chaîne 1D périodique de 109 qubits, qui est la plus longue chaîne 1D dans la topologie d'un dispositif IBM Quantum à 127 qubits. Il n'est pas possible de disposer une chaîne périodique de 109 qubits sur un dispositif de 127 qubits de sorte que les premier et dernier qubits soient voisins sans incorporer des portes SWAP supplémentaires.

In [2]:
init_layout = [
    13,
    12,
    11,
    10,
    9,
    8,
    7,
    6,
    5,
    4,
    3,
    2,
    1,
    0,
    14,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    36,
    51,
    50,
    49,
    48,
    47,
    46,
    45,
    44,
    43,
    42,
    41,
    40,
    39,
    38,
    37,
    52,
    56,
    57,
    58,
    59,
    60,
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    74,
    89,
    88,
    87,
    86,
    85,
    84,
    83,
    82,
    81,
    80,
    79,
    78,
    77,
    76,
    75,
    90,
    94,
    95,
    96,
    97,
    98,
    99,
    100,
    101,
    102,
    103,
    104,
    105,
    106,
    107,
    108,
    112,
    126,
    125,
    124,
    123,
    122,
    121,
    120,
    119,
    118,
    117,
    116,
    115,
    114,
    113,
]

# the number of qubits in the circuit is governed by the length of the initial layout
num_qubits = len(init_layout)
num_qubits

109

#### Build the entangler map for the TwoLocal circuit

In [3]:
coupling_map = [(i, i + 1) for i in range(0, len(init_layout) - 1)]
coupling_map.append(
    (len(init_layout) - 1, 0)
)  # adding in the periodic connectivity

#### Construction de la carte d'intrication pour le circuit TwoLocal

In [4]:
num_reps = 2
entangler_map = []

for even_edge in coupling_map[0 : len(coupling_map) : 2]:
    entangler_map.append(even_edge)

for odd_edge in coupling_map[1 : len(coupling_map) : 2]:
    entangler_map.append(odd_edge)

In [ ]:
ansatz = TwoLocal(
    num_qubits=num_qubits,
    rotation_blocks="rx",
    entanglement_blocks="cx",
    entanglement=entangler_map,
    reps=num_reps,
).decompose()
ansatz.draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/periodic-boundary-conditions-with-circuit-cutting/extracted-outputs/79428537-66cf-40ce-87cf-0f75f591cb4b-0.avif" alt="Output of the previous code cell" />

In order to verify the quality of the outcome using circuit cutting, we need to know the ideal outcome. The current circuit of choice is beyond brute force classical simulation. Therefore, we fix the parameters to the circuit carefully to make it clifford.

We shall assign the parameter value $0$ for the first two layers of `Rx` gates, and the value $\pi$ for the last layer. This ensures that the ideal outcome of this circuit is $|1\rangle^{\otimes n}$, $n$ being the number of qubits. Therefore, the expectation values of $\langle Z_i \rangle$ and $\langle Z_i Z_{i+1} \rangle$, where $i$ is the index of the qubit, are $-1$ and $+1$ respectively.

In [ ]:
params_last_layer = [np.pi] * ansatz.num_qubits
params = [0] * (ansatz.num_parameters - ansatz.num_qubits)
params.extend(params_last_layer)

ansatz.assign_parameters(params, inplace=True)

![Output of the previous code cell](../docs/images/tutorials/periodic-boundary-conditions-with-circuit-cutting/extracted-outputs/79428537-66cf-40ce-87cf-0f75f591cb4b-0.avif)

Afin de vérifier la qualité du résultat obtenu par découpe de circuits, nous devons connaître le résultat idéal. Le circuit actuel dépasse les capacités de simulation classique par force brute. Par conséquent, nous fixons soigneusement les paramètres du circuit pour le rendre clifford.

Nous assignerons la valeur de paramètre $0$ pour les deux premières couches de portes `Rx`, et la valeur $\pi$ pour la dernière couche. Cela garantit que le résultat idéal de ce circuit est $|1\rangle^{\otimes n}$, $n$ étant le nombre de qubits. Par conséquent, les valeurs d'espérance de $\langle Z_i \rangle$ et $\langle Z_i Z_{i+1} \rangle$, où $i$ est l'indice du qubit, sont respectivement $-1$ et $+1$.

In [ ]:
observables = []

for i in range(num_qubits):
    obs = "I" * (i) + "Z" + "I" * (num_qubits - i - 1)
    observables.append(obs)

for i in range(num_qubits):
    if i == num_qubits - 1:
        obs = "Z" + "I" * (num_qubits - 2) + "Z"
    else:
        obs = "I" * i + "ZZ" + "I" * (num_qubits - i - 2)
    observables.append(obs)

observables = SparsePauliOp(observables)
paulis = observables.paulis
coeffs = observables.coeffs

#### Sélection des observables
Pour quantifier les bénéfices du gate cutting, nous mesurons les valeurs d'espérance des observables $\frac{1}{n}\sum_{i=1}^n \langle Z_i \rangle$ et $\frac{1}{n-1}\sum_{i=1}^{n-1} \langle Z_i Z_{i+1} \rangle$. Comme discuté précédemment, les valeurs d'espérance idéales sont respectivement $-1$ et $+1$.

In [8]:
coupling_map = backend.configuration().coupling_map

# create a virtual coupling map with long range connectivity
virtual_coupling_map = coupling_map.copy()
virtual_coupling_map.append([init_layout[-1], init_layout[0]])
virtual_coupling_map.append([init_layout[0], init_layout[-1]])

In [ ]:
pm_virtual = generate_preset_pass_manager(
    optimization_level=1,
    coupling_map=virtual_coupling_map,
    initial_layout=init_layout,
    basis_gates=backend.configuration().basis_gates,
)

virtual_mapped_circuit = pm_virtual.run(ansatz)
virtual_mapped_circuit.draw("mpl", fold=-1, idle_wires=False)

<Image src="../docs/images/tutorials/periodic-boundary-conditions-with-circuit-cutting/extracted-outputs/ad38aa32-4613-46c5-bf62-da332a1b9dfb-0.avif" alt="Output of the previous code cell" />

#### Cut the long range periodic connectivities

Now we cut the gates in the transpiled circuit. Note that the 2-qubit gates that need to be cut are the ones connecting the last and the first qubits of the layout.

In [ ]:
# Find the indices of the distant gates
cut_indices = [
    i
    for i, instruction in enumerate(virtual_mapped_circuit.data)
    if {virtual_mapped_circuit.find_bit(q)[0] for q in instruction.qubits}
    == {init_layout[-1], init_layout[0]}
]

![Output of the previous code cell](../docs/images/tutorials/periodic-boundary-conditions-with-circuit-cutting/extracted-outputs/ad38aa32-4613-46c5-bf62-da332a1b9dfb-0.avif)

#### Découpe des connectivités périodiques longue distance
Nous découpons maintenant les portes dans le circuit transpilé. Notez que les portes à 2 qubits qui doivent être découpées sont celles connectant les dernier et premier qubits du placement.

In [12]:
trans_observables = observables.apply_layout(virtual_mapped_circuit.layout)

Nous allons appliquer le placement du circuit transpilé à l'observable.

In [ ]:
qpd_circuit, bases = cut_gates(virtual_mapped_circuit, cut_indices)
subexperiments, coefficients = generate_cutting_experiments(
    circuits=qpd_circuit,
    observables=trans_observables.paulis,
    num_samples=np.inf,
)

Enfin, les sous-expériences sont générées en échantillonnant sur différentes bases de mesure et de préparation.

In [14]:
print(f"Number of subexperiments is {len(subexperiments)} = 6**{num_reps}")

Number of subexperiments is 36 = 6**2


Notez que la découpe des interactions longue distance entraîne l'exécution de multiples échantillons du circuit qui diffèrent par les bases de mesure et de préparation. Plus d'informations à ce sujet sont disponibles dans <a href='https://arxiv.org/abs/1909.07534'>Constructing a virtual two-qubit gate by sampling single-qubit operations</a> et <a href='https://arxiv.org/abs/2312.11638'>Cutting circuits with multiple two-qubit unitaries</a>.

Le nombre de portes périodiques à découper est égal au nombre de répétitions de la couche `TwoLocal`, défini par `num_reps` ci-dessus. Le surcoût d'échantillonnage du gate cutting est de 6. Par conséquent, le nombre total de sous-expériences sera $6^{num\_reps}$.

In [ ]:
pass_ = PassManager(
    [Optimize1qGatesDecomposition(basis=backend.configuration().basis_gates)]
)

subexperiments = pass_.run(
    [
        dag_to_circuit(
            BasisTranslator(sel, target_basis=backend.basis_gates).run(
                circuit_to_dag(circ)
            )
        )
        for circ in subexperiments
    ]
)

## Step 3: Execute using Qiskit primitives

<ul>
    <li>Input: Target circuits</li>
    <li>Output: Quasi-probability distributions</li>
</ul>

We use a `SamplerV2` primitive for execution of the cut circuits. We disable `dynamical decoupling` and `twirling` so that any improvement we obtain in the result will solely be due to effective application of gate cutting for this type of circuit.

In [ ]:
options = SamplerOptions()
options.default_shots = 10000
options.dynamical_decoupling.enable = False
options.twirling.enable_gates = False
options.twirling.enable_measure = False

#### Transpilation des sous-expériences
À ce stade, les sous-expériences contiennent des circuits avec certaines portes à 1 qubit qui ne font pas partie du jeu de portes de base. Cela est dû au fait que les qubits découpés sont mesurés dans différentes bases, et les portes de rotation utilisées à cet effet n'appartiennent pas nécessairement au jeu de portes de base. Par exemple, la mesure dans la base X implique l'application d'une porte de Hadamard avant la mesure habituelle dans la base Z. Mais la porte de Hadamard ne fait pas partie du jeu de portes de base.

Au lieu d'appliquer l'ensemble du processus de transpilation sur chacun des circuits des sous-expériences, nous pouvons utiliser des passes de transpilation spécifiques. Consultez <a href="https://docs.quantum.ibm.com/api/qiskit/transpiler_passes">cette documentation</a> pour une description détaillée de toutes les passes de transpilation disponibles.

Nous allons appliquer les passes ```BasisTranslator``` puis ```Optimize1qGatesDecomposition``` pour nous assurer que toutes les portes de ces circuits appartiennent au jeu de portes de base. L'utilisation de ces deux passes est plus rapide que l'ensemble du processus de transpilation, puisque d'autres étapes telles que le routage et la sélection du placement initial ne sont pas effectuées à nouveau.

In [ ]:
with Batch(backend=backend) as batch:
    sampler = SamplerV2(options=options)
    cut_job = sampler.run(subexperiments)

print(f"Job ID {cut_job.job_id()}")

Job ID cwxf7wq60bqg008pvt8g


In [18]:
result = cut_job.result()

Nous allons maintenant soumettre les tâches en mode batch.

In [ ]:
reconstructed_expvals = reconstruct_expectation_values(
    result,
    coefficients,
    paulis,
)

We now calculate the average of weight-1 and weight-2 Z-type observables.

In [20]:
cut_weight_1 = np.mean(reconstructed_expvals[:num_qubits])
cut_weight_2 = np.mean(reconstructed_expvals[num_qubits:])

print(f"Average of weight-1 expectation values is {cut_weight_1}")
print(f"Average of weight-2 expectation values is {cut_weight_2}")

Average of weight-1 expectation values is -0.741733944954063
Average of weight-2 expectation values is 0.6968862385320495


### Cross Verify: Obtain uncut expectation value

It is useful to cross-verify the advantage of the circuit cutting technique against uncut. Here we shall compute the expectation values without cutting the circuit. Note that such an uncut circuit will suffer from a large number of SWAP gates required to implement the 2-qubit operation between the first and the last qubits. We shall use the `sampled_expectation_value` function to obtain the expectation values of the uncut circuit after obtaining the probability distribution via `SamplerV2`. This allows a homogenous usage of primitive over all the instances. However, note that we could have used `EstimatorV2` as well to directly compute the expectation values.

In [23]:
if ansatz.num_clbits == 0:
    ansatz.measure_all()

pm_uncut = generate_preset_pass_manager(
    optimization_level=1, backend=backend, initial_layout=init_layout
)

transpiled_circuit = pm_uncut.run(ansatz)

In [24]:
sampler = SamplerV2(mode=backend, options=options)
uncut_job = sampler.run([transpiled_circuit])

In [25]:
uncut_job_id = uncut_job.job_id()
print(f"The job id for the uncut clifford circuit is {uncut_job_id}")

The job id for the uncut clifford circuit is cwxfads2ac5g008jhe7g


In [26]:
uncut_result = uncut_job.result()[0]
uncut_counts = uncut_result.data.meas.get_counts()

### Vérification croisée : Obtenir les valeurs d'espérance sans découpe
Il est utile de vérifier l'avantage de la technique de découpe de circuits par rapport à l'absence de découpe. Ici, nous allons calculer les valeurs d'espérance sans découper le circuit. Notez qu'un tel circuit non découpé souffrira d'un grand nombre de portes SWAP nécessaires pour implémenter l'opération à 2 qubits entre les premier et dernier qubits. Nous utiliserons la fonction `sampled_expectation_value` pour obtenir les valeurs d'espérance du circuit non découpé après avoir obtenu la distribution de probabilité via `SamplerV2`. Cela permet une utilisation homogène de la primitive sur toutes les instances. Notez cependant que nous aurions également pu utiliser `EstimatorV2` pour calculer directement les valeurs d'espérance.

In [ ]:
uncut_expvals = [
    sampled_expectation_value(uncut_counts, obs) for obs in paulis
]

uncut_weight_1 = np.mean(uncut_expvals[:num_qubits])
uncut_weight_2 = np.mean(uncut_expvals[num_qubits:])

print(f"Average of weight-1 expectation values is {uncut_weight_1}")
print(f"Average of weight-2 expectation values is {uncut_weight_2}")

Average of weight-1 expectation values is -0.32494128440366965
Average of weight-2 expectation values is 0.32340917431192656


### Visualize

Let us now visualize the improvement obtained for weight-1 and weight-2 observables when using gate cutting for periodic chain circuit

In [ ]:
mpl.rcParams.update(mpl.rcParamsDefault)

fig = plt.subplots(figsize=(12, 8), dpi=200)
width = 0.25
labels = ["Weight-1", "Weight-2"]
x = np.arange(len(labels))

ideal = [-1, 1]
cut = [cut_weight_1, cut_weight_2]
uncut = [uncut_weight_1, uncut_weight_2]

br1 = np.arange(len(ideal))
br2 = [x + width for x in br1]
br3 = [x + width for x in br2]

plt.bar(
    br1, ideal, width=width, edgecolor="k", label="Ideal", color="#4589ff"
)
plt.bar(br2, cut, width=width, edgecolor="k", label="Cut", color="#a56eff")
plt.bar(
    br3, uncut, width=width, edgecolor="k", label="Uncut", color="#009d9a"
)

plt.axhline(y=0, color="k", linestyle="-")

plt.xticks([r + width for r in range(len(ideal))], labels, fontsize=14)
plt.yticks(fontsize=14)

plt.legend(fontsize=14)
plt.show()

<Image src="../docs/images/tutorials/periodic-boundary-conditions-with-circuit-cutting/extracted-outputs/2ba8913f-ba35-409c-bc4c-5f28e3698f20-0.avif" alt="Output of the previous code cell" />

### Summary

In summary, we calculated the average expectation values of weight-1 and weight-2 Z-types observables for a periodic 1D chain of 109 qubits. In order to do so, we

- created a virtual coupling map by adding a long range connectivity between the first and the last qubits of the 1D chain, and transpiled the circuit.
    - transpilation at this stage allowed us to avoid the overhead of transpiling each subexperiment separately after cutting,
    - using virtual coupling map allowed us to avoid extra SWAP gates for the 2-qubit operation between the first and the last qubits.
- removed the long range connectivity from the transpiled circuit via gate cutting.
- converted the cut circuits into basis gate set by applying appropriate transpilation passes.
- executed the cut circuits on IBM Quantum device using a `SamplerV2` primitive.
- obtained the expectation value by reconstructing the outcomes of the cut circuits.

### Inference

We notice from the results that the average of the weight-1 $\langle Z \rangle$ and weight-2 $\langle ZZ \rangle$ type observables are significantly improved by cutting the periodic gates. Note that this study does not include any error suppression or mitigation techniques. The improvement observed is solely due to the proper usage of gate cutting for this problem. The results could have been further improved by using the mitigation and suppression techniques.

This study shows an example of effectively using gate cutting to improve the performance of computation.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3fQQYAIjTxvIChg)